## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
from keras.optimizers import Adam
from keras.layers import Input, GRU, Flatten, MaxPool2D, MaxPool1D
from keras.layers import PReLU, Dropout, Lambda, Dense
from keras.models import Model
import tensorflow as tf
from scipy.signal import stft

from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis import preprocessing

from music_generator.musical import scales
import numpy as np
from multiprocessing import Pool
from functools import partial

import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline
import matplotlib

from scipy.io.wavfile import read
import pandas as pd

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.rcParams['lines.linewidth'] = 2
matplotlib.rcParams['axes.linewidth'] = 1.5
matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['xtick.major.size'] = 5
matplotlib.rcParams['xtick.major.width'] = 2
matplotlib.rcParams['ytick.major.size'] = 5
matplotlib.rcParams['ytick.major.width'] = 2
matplotlib.rcParams['figure.figsize'] = (16.0, 8.0)

## Load & show & play music

In [ ]:
from scipy.io.wavfile import read

In [ ]:
sr, data = read('../data/full-mix.wav')

In [ ]:
data = pd.Series(data)
data.index = data.index / sr
data = data / 2**15 
# data = data.set_index('time')

In [ ]:
data.plot();

In [ ]:
plt.plot(data.loc[40:50])
Audio(data.loc[40:50].values, rate=sr)

In [ ]:
plt.plot(data.loc[40:41])
Audio(data.loc[40:41].values, rate=sr)

In [ ]:
plt.plot(data.loc[40.8:41])
Audio(data.loc[40.8:41].values, rate=sr)

## Simple synthesizer: tone generation

### Sine generator

In [ ]:
generated = pd.DataFrame({'time': np.arange(0, 1, 1/sr)}).set_index('time')
generated['sine'] = np.sin(generated.index * 440 * 2 * np.pi)

In [ ]:
def plot_and_play(df, col_name):
    fig, ax = plt.subplots(ncols=2)

    plt.sca(ax[0])
    plt.title('All data')
    df[col_name].plot()

    plt.sca(ax[1])
    plt.title('Zoomed in')
    df.loc[0:0.02][col_name].plot()
    
    plt.show()
    
    fig, ax = plt.subplots()
    plt.title('Spectral')
    f_vec, t_vec, Zxx = stft(df[col_name], sr, nperseg=2048, noverlap=2048 // 4)
    plt.pcolormesh(t_vec, f_vec, np.abs(Zxx)) #, vmin=0, vmax=np.percentile(np.abs(Zxx), 99))
    # plt.title('STFT Magnitude')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.ylim([0, 5000])
    
    plt.show()

    return Audio(df[col_name], rate=sr)

In [ ]:
plot_and_play(generated, 'sine')

### Envelope

In [ ]:
dr = 0.2
generated['decay_envelope'] = np.exp(-generated.index / dr)
generated['decay_sine'] = generated['sine'] * generated['decay_envelope']

In [ ]:
plot_and_play(generated, 'decay_sine')

### Additive synthesis

In [ ]:
def additive_synthesis(t, amps, freqs):
    return np.sum([amp * np.sin(t * freq * 2 * np.pi) for amp, freq in zip(amps, freqs)], axis=0)

In [ ]:
dr = 0.2

amps = [1, -0.3, 0.1, -0.1, 0.4, 0.01, -0.2]
freqs = [440, 2*440, 3*440, 4*440, 5*440, 6*440, 7*440]

generated['additive'] = additive_synthesis(generated.index, amps, freqs)
generated['additive_decay'] = generated['additive'] * generated['decay_envelope']

In [ ]:
plot_and_play(generated, 'additive')

In [ ]:
plot_and_play(generated, 'additive_decay')

### Drums

In [ ]:
generated['kick'] = np.sin( (1 / (generated.index + 0.1) + 30 ) * 2 * np.pi)
generated['kick'] *= np.exp(-generated.index / 0.4)
plot_and_play(generated, 'kick')

### Snare

In [ ]:
generated['noise'] = np.random.uniform(low=-1, high=1, size=generated.index.shape)
plot_and_play(generated, 'noise')

In [ ]:
generated['short_decay'] = np.exp(-generated.index / 0.05)
generated['snare_base'] = np.sin( (1 / (generated.index + 0.1) + 30 ) * 4 * np.pi) * np.exp(-generated.index / 0.4)
generated['snare'] = generated['snare_base'] + generated['noise'] * generated['short_decay']
plot_and_play(generated, 'snare')

### Subtractive synthesis

In [ ]:
generated['square'] = np.sign(np.sin(generated.index * 440 * 4)) * generated['decay_envelope']
plot_and_play(generated, 'square')

In [ ]:
from music_generator.basic.signalproc import apply_filter

In [ ]:
generated['filtered_square'] = apply_filter(generated['square'].values, SamplingInfo(sr), 3000, order=5, type='lowpass')

In [ ]:
plot_and_play(generated, 'filtered_square')